In [1]:
import pandas as pd
import numpy as np

In [4]:
sixteenS_v4 = pd.read_csv('../data/cleaned/NCOG_21_16sV4_redo2_filtered_asv_count_tax.tsv', sep='\t')
sixteenS_v4.set_index('Unnamed: 0', drop=True, inplace=True)
sixteenS_v4.index = sixteenS_v4.index.rename('sample')
eighteenS_v4 = pd.read_csv('../data/cleaned/NCOG_18sV4_filtered_asv_count_tax.tsv', sep='\t')
eighteenS_v4.set_index('Unnamed: 0', drop=True, inplace=True)
eighteenS_v4.index = eighteenS_v4.index.rename('sample')
meta_v4 = pd.read_csv('../data/cleaned/NCOG_v4_sample_log_DNA_stvx_meta_filtered_2014-2020_mod.tsv', sep='\t')
meta_v4.set_index('Unnamed: 0', drop=True, inplace=True)
meta_v4.index = meta_v4.index.rename('sample')

In [5]:
df = pd.concat([sixteenS_v4, eighteenS_v4], axis=1)

### Below takes ~30min

In [6]:
# add scale on bottom to show the range of the nodes that were removed

In [8]:
env_of_interest = ['Depthm', 'T_degC', 'Salnty', 'STheta', 'O2ml_L', 'PO4ug', 'SiO3ug', 'NO3ug', 'NH3ug', 'ChlorA',
                   'Phaeop', 'MLD_Sigma', 'NCDepth']
tol_ranges = {env: {'otus': df.columns.tolist(), '25': [], '50': [], '75': []} for env in env_of_interest}
for i, otu in enumerate(df.columns):
    print(roun100/len(df.columns), 2), end='\r')
    #if np.mean(df[otu] > 0) > 0.30:
    # calculate tolerance range of each metadata
    otu_sum = np.sum(df[otu])
    if otu_sum > 0:
        otu_envs = [[] for x in env_of_interest]
        for sample, entry in df[otu].items():
            if entry > 0:
                # 10000 * (number of otus in sample / total number of otus in all samples)
                count = 10000 * (entry / otu_sum)
                for j in range(round(count)):
                    for k in range(len(otu_envs)):
                        otu_envs[k].append(meta_v4.loc[sample, env_of_interest[k]])
        for i, env in enumerate(env_of_interest):
            tol_ranges[env]['25'].append(np.quantile(otu_envs[i], 0.25))
            tol_ranges[env]['50'].append(np.median(otu_envs[i]))
            tol_ranges[env]['75'].append(np.quantile(otu_envs[i], 0.75))
    else:
        for i, env in enumerate(env_of_interest):
            tol_ranges[env]['25'].append(np.nan)
            tol_ranges[env]['50'].append(np.nan)
            tol_ranges[env]['75'].append(np.nan)
#     else:
#         for env in env_of_interest:
#             tol_ranges[env]['25'].append(np.nan)
#             tol_ranges[env]['50'].append(np.nan)
#             tol_ranges[env]['75'].append(np.nan)
print()

with pd.ExcelWriter("../data/tolerance_ranges/OTU_metavar_tolerance_ranges.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    for key in tol_ranges.keys():
        pd.DataFrame(tol_ranges[key]).set_index('otus', drop=True).to_excel(writer, sheet_name=key)

99.97
